In [2]:
import pandas as pd
import numpy as np

data = pd.read_csv('/home/nika/code/marcnaweb/car_recommendation_engine/raw_data/car_files_4c_en.csv')
df = data.copy()
df.head()

/tmp/ipykernel_43338/3543565571.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/tmp/ipykernel_43338/3543565571.py:4: DtypeWarning: Columns (24,25,26,35,43,53,59,63,74,75,85,89,108,110,116,120,122,124,136,141,147,150,151,162,163,177,178,180,195,214,221,226,234,241,246,259,266,270,271,273,278,283,287,290) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/home/nika/code/marcnaweb/car_recommendation_engine/raw_data/car_files_4c_en.csv')


,Unnamed: 0.1,Unnamed: 0,Hot air,Volumetric anti -first alarm,Harman Kardon Sound System,Track change alert,Carbonaceramic brake disc,Arconditional Zones 2,Reclinarinar rear seat,Maximum rotation,...,Folding rear seat,Free wheel,Shaker Pro Audio Sound System,Maximum speed electric mode,Electronic noise cancellation,Steering wheel adjustment in depth,Facial recognition camera,Carcode,Perimeter anti theft alarm,Track centralization assistant
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN


## Too many features with NaN's, remove columns which have more then 15000 Nan'n in column

In [3]:
filtered_columns = df.columns[df.isna().sum() < 15000]
filtered_df = df[filtered_columns]
# filtered_df

## Drop some rows with a lot of Nan's

In [4]:
test_filtered_df =filtered_df.dropna(subset='Acceleration')
# test_filtered_df

## Check how much % is NaN in every column and check Dtype

In [5]:
nan_percentage_per_column = test_filtered_df.isnull().mean() * 100

# Create a new DataFrame with column names and NaN percentages and Dtypes
nan_info_df = pd.DataFrame({
    'Column Name': nan_percentage_per_column.index,
    'NaN Percentage': nan_percentage_per_column.values,
    'Data Type': test_filtered_df.dtypes
})

# Display the new DataFrame
nan_info_df # To check whole list: nan_info_df[0:50] and nan_info_df[50:100]

,Column Name,NaN Percentage,Data Type
Unnamed: 0.1,Unnamed: 0.1,0.000000,int64
Hot air,Hot air,46.233340,object
Acceleration,Acceleration,0.000000,object
Length,Length,0.000000,object
Maximum torque,Maximum torque,0.709871,object
...,...,...,...
Year,Year,0.125555,float64
Radio,Radio,49.565385,object
Folding rear seat,Folding rear seat,52.796021,object
Carcode,Carcode,0.000000,int64


## After analysing all features, decided to remove some of them.
Neither they don't give information, or they are hard to preprocess.


In [6]:
column_to_remove = ['Unnamed: 0.1','Assistance','Carcode','Aspiration', 'Rear tires', 'Spare tire', 'Front tires',
                    'Urban autonomy', 'Generation', 'Front suspension', 'Rear suspension', 'Coupling', 'Valve command',
                    'Places', 'Disposition', 'Cylinders', 'Elastic element', 'Ipva R', 'Frontal area A', 'Engine code',
                    'Traction', 'Installation', 'Road autonomy', 'Engine power supply', 'Engine control activation',
                    'Gear change code', 'Corrected frontal area', 'Platform']


In [7]:
new_filtered_df = test_filtered_df.drop(columns=column_to_remove)
len(new_filtered_df.columns)

71

# Preprocessing Features

## Preprocessing All features with [Nan, standart equipment, optinal equipment]
saved in: columns_to_transform ---------
Nan = 0 ----------------
standart equipment, optinal equipment = 1

In [8]:
preprocessing_df = new_filtered_df.copy()

columns_to_transform = ['Hot air', 'Rev counter', 'Assisted direction', 'ABS brakes', 'Rear window', 'Central locking of the doors',
                        'Headrest for all occupants', 'Electric rearview mirror adjustment', 'Air conditioning',
                        'Bluetooth connection', 'Frontal Airbags', 'Steering wheel adjustment height',
                        'Electric front window control', 'Multifunctional steering wheel', 'Driver s seat with height adjustment',
                        'On board computer', 'Light in the trunk', 'Alloy wheels', 'USB connection', 'Radio',
                        'Folding rear seat', 'Perimeter anti theft alarm', 'Cooling liquid thermometer']

# Replace NaN with 0 and other values with 1
preprocessing_df[columns_to_transform] = preprocessing_df[columns_to_transform].applymap(lambda x: 0 if pd.isnull(x) else 1)


/tmp/ipykernel_43338/589099134.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  preprocessing_df[columns_to_transform] = preprocessing_df[columns_to_transform].applymap(lambda x: 0 if pd.isnull(x) else 1)


## Preprocessing [Acceleration]

In [9]:
# Preprocess Acceleration (0100 km/h 3,8 s = 3.8)
preprocessing_df['Acceleration'] = preprocessing_df['Acceleration'].str.extract(r'(\d+\,\d+)')

# Replace ',' with '.' and convert to numeric
preprocessing_df['Acceleration'] = preprocessing_df['Acceleration'].str.replace(',', '.').astype(float)
# Rename Column
# preprocessing_df = preprocessing_df.rename(columns={'Acceleration': 'Acceleration 0100 km/h in S'})

## Preprocessing [All features with strings]

In [10]:
# Rename columns
# preprocessing_df = preprocessing_df.rename(columns={'Length': 'Length mm',
#                                                     'Width': 'Width mm',
#                                                     'Maximum torque': 'Maximum torque kgfm'})

In [11]:
def extract_float_value(value):
    try:
        if isinstance(value, float):
            return value
        else:
            float_value = value.split()[0].replace(',', '.')
            return float(float_value)
    except (ValueError, IndexError):
        return np.nan

for column in preprocessing_df[['Weight/Torque', 'Weight', 'Weight/power', 'Max power regime.', 'Cylinder diameter',
                      'Fuel tank', 'Specific power', 'Maximum power', 'Length', 'Maximum torque', 'Width', 'Height',
                      'Specific torque', 'Minimum height from the ground', 'Piston course', 'Front gauge', 'Displacement',
                      'Turns diameter', 'Rear gauge', 'Length between the axis', 'Maximum speed', 'Road consumption',
                      'Max torque regime', 'Car payload', 'Sidewall height', 'Unit displacement', 'Trunk', 'Urban',
                      'Guarantee', 'Reader score', 'Compression ratio', 'Drag coefficient', 'Price R', 'Devaluation',
                      'CNW Index']]:
    preprocessing_df[column] = preprocessing_df[column].apply(extract_float_value)


In [12]:
len(preprocessing_df.columns)

71

In [76]:
preprocessing_df

,Hot air,Acceleration,Length,Maximum torque,Width,Rev counter,Compression ratio,Assisted direction,CNW Ranking,ABS brakes,...,Car size,Unit displacement,Trunk,USB connection,Gear speed transmissions,Urban,Year,Radio,Folding rear seat,Perimeter anti theft alarm
0,0,3.8,4580.0,66.3,2045.0,0,10.71,0,20549.0,0,...,Grande,516.0,50.0,0,6.0,2.4,2002.0,0,0,0
1,0,15.2,3644.0,8.5,1548.0,0,9.51,0,14268.0,0,...,Compact and portable,250.0,290.0,0,5.0,10.0,2002.0,0,0,0
2,0,10.6,4177.0,17.3,1709.0,0,9.81,0,12266.0,0,...,Medium,500.0,370.0,0,5.0,7.2,2002.0,0,0,0
4,0,8.5,4149.0,21.4,1735.0,0,9.51,0,16041.0,0,...,Medium,445.0,330.0,0,5.0,6.8,2002.0,0,0,0
5,0,9.5,4187.0,16.7,1741.0,0,10.31,0,9142.0,0,...,Medium,437.0,370.0,0,5.0,9.0,2002.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23881,1,10.7,4015.0,17.5,1720.0,1,10.51,1,63.0,1,...,Compact and portable,333.0,300.0,1,6.0,8.3,2024.0,1,1,1
23882,1,10.7,4015.0,17.5,1720.0,1,10.51,1,54.0,1,...,Compact and portable,333.0,300.0,1,6.0,8.3,2024.0,1,1,1
23890,1,10.7,4015.0,17.5,1720.0,1,10.51,1,50.0,1,...,Compact and portable,333.0,300.0,1,6.0,9.0,2024.0,1,1,1
23902,1,14.9,5413.0,34.7,2050.0,1,161.00,1,1073.0,1,...,Grande,545.0,11500.0,1,6.0,10.0,2023.0,1,0,1


## OneHotEncode categorical features

In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_selector

In [37]:
columns_to_encode = ['Propulsion', 'Car gearbox', 'Fuel', 'Settings', 'Car size']

# Create Pipeline
pipeline_onehot = Pipeline([
    ('encoder', ColumnTransformer([
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False), make_column_selector(dtype_include="object"))]  # Apply one-hot encoding to the specified columns
    , remainder='passthrough'))  # Pass through any remaining columns unchanged
])

In [64]:
# create dataframe with features we want to encode
onehotencoded_df = preprocessing_df[columns_to_encode]
# Fit and transform the data
onehotencoded_df = pipeline_onehot.fit_transform(onehotencoded_df)
# Get column names
encoded_column_names = pipeline_onehot.named_steps['encoder'].get_feature_names_out(columns_to_encode)
# Remove onehot__ from the columns (example: onehot__Propulsion_combustion TRANSFORM: Propulsion_combustion )
encoded_column_names = [column.replace('onehot__', '') for column in encoded_column_names]
# Transform to pd dataframe with column names
onehotencoded_df = pd.DataFrame(onehotencoded_df, columns = encoded_column_names)
# Remove columns that we encoded from main df
remaining_columns_df = preprocessing_df.drop(columns=columns_to_encode)
# Combine the one-hot encoded features with the remaining columns of the original DataFrame
final_df = pd.concat([remaining_columns_df, onehotencoded_df], axis=1)
final_df.columns

len(final_df.columns)


# We had 71 columns at the start, we encoded 5 columns and as a result we received 35 columns,
# 71+35 = 106. now we delete original 5 columns, so 106-5=101. Everything is correct.


101

In [91]:
# preprocessing_df

In [90]:
final_df.shape[]

SyntaxError: invalid syntax (4164918896.py, line 1)

In [68]:
test_df = final_df.dropna(subset=['Hot air'])
test_df

,Hot air,Acceleration,Length,Maximum torque,Width,Rev counter,Compression ratio,Assisted direction,CNW Ranking,ABS brakes,...,Settings_SUV,Settings_Sedan,"Settings_Turkey, hen",Settings_Van,Car size_Compact and portable,Car size_Coupe,Car size_Grande,Car size_Medium,Car size_Medium Compact,Car size_Subcompact
0,0.0,3.8,4580.0,66.3,2045.0,0.0,10.71,0.0,20549.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,15.2,3644.0,8.5,1548.0,0.0,9.51,0.0,14268.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,10.6,4177.0,17.3,1709.0,0.0,9.81,0.0,12266.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,8.5,4149.0,21.4,1735.0,0.0,9.51,0.0,16041.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,0.0,9.5,4187.0,16.7,1741.0,0.0,10.31,0.0,9142.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23881,1.0,10.7,4015.0,17.5,1720.0,1.0,10.51,1.0,63.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23882,1.0,10.7,4015.0,17.5,1720.0,1.0,10.51,1.0,54.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23890,1.0,10.7,4015.0,17.5,1720.0,1.0,10.51,1.0,50.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23902,1.0,14.9,5413.0,34.7,2050.0,1.0,161.00,1.0,1073.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
final_df.isna()
nan_percentage = (test_df.isna().sum() / len(test_df)) * 100
nan_percentage[0:20]


Hot air                            0.000000
Acceleration                      17.046552
Length                             0.000000
Maximum torque                     0.709871
Width                              0.000000
Rev counter                        0.000000
Compression ratio                  1.265212
Assisted direction                 0.000000
CNW Ranking                        0.000000
ABS brakes                         0.000000
Rear window                        0.000000
Tuching                           42.080355
Weight/Torque                      0.000000
Guarantee                          0.000000
Drag coefficient                  37.328569
Central locking of the doors       0.000000
Height                             0.000000
Valves per cylinder                0.714700
Specific torque                    0.709871
Minimum height from the ground    45.484837
dtype: float64